### **Fundamentos de Aprendizaje Automático**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Práctica 1: Naive Bayes**

## Autores:  
Diego Araque Fernández  
Angela Valderrama Ricaldi

# Introducción

Esta práctica consta de dos apartados relacionados con la clasificación Naive Bayes. En el primer apartado, se implemente nuestra propia versión del clasificador, considerando la opción de aplicar la corrección de Laplace. En el segundo apartado, se utiliza la implementación de Naive Bayes de la librería scikit-learn. El objetivo de esta práctica es comparar los resultados obtenidos en ambos apartados.

# 1. Naive-Bayes propio

La clase abstracta Clasificador define la interfaz a seguir de los diferentes clasificadores que se implementarán a lo largo de las siguientes prácticas. En este caso, se implementa la clase ClasificadorNaiveBayes, que hereda de Clasificador y que tendrá que implementar los métodos entrenamiento y clasifica.

Como ya sabemos, Naive Bayes supone que los atributos son independientes entre sí y la regla de clasificación es la siguiente:

$$C_{MAP} = \underset{C_i}{\operatorname{argmax}} P(C_i) \prod_{j=1}^{n} P(A_j|C_i)$$

siendo $C_{MAP}$ la clase que maximiza la probabilidad a posteriori, $P(C_i)$ la probabilidad a priori de la clase $C_i$ y $P(A_j|C_i)$ la probabilidad condicionada de que el atributo $A_j$ pertenezca a la clase $C_i$.

Distinguimos dos métodos abstractos en la clase Clasificador: entrenamiento y clasifica. El método entrenamiento se encarga de entrenar el clasificador, es decir, de calcular las probabilidades de que un ejemplo pertenezca a cada una de las clases (prioris). El método clasifica se encarga de clasificar un ejemplo, es decir, de asignarle una clase determinada (posteriori).

Para el clasificador Naive Bayes, el método _entrenamiento()_ calcula las probabilidades a priori de cada clase con el conjunto de datos de entrenamiento. Para ello, se calcula el número de ejemplos de cada clase y se divide entre el número total de ejemplos:

$$P(C_i) = \frac{N_i}{N}$$

Este valor coincide con la frecuencia relativa de cada clase. Además, se calculan las verosimilitudes de cada atributo para cada clase, es decir, las probabilidades condicionadas de cada atributo para cada clase:

$$P(A_j|C_i) = \frac{N_{ij}}{N_i}$$

En los dos datasets que tenemos: ***Tic-Tac-Toe*** y ***Heart***, nos encontramos con atributos nominales y numéricos. Según el teorema de Bayes, para calcular las verosimilitudes de un atributo nominal, se calcula la frecuencia relativa de dicho atributo para cada clase como en la fórmula anterior.

Puede darse el caso en el que no hay ocurrencias de un atributo para una clase determinada. En ese caso, la probabilidad condicionada de dicho atributo para esa clase será 0, lo que hará que la probabilidad a posteriori de esa clase sea 0 y se descarte como posible clase para el ejemplo a clasificar. Para evitar esto, se permite el uso de la corrección de Laplace con el argumento `laplace` al inicializar el clasificador. Esta corrección consiste en sumar 1 al numerador y el número de valores que puede tomar el atributo al denominador de la fórmula anterior:

$$P(A_j|C_i) = \frac{N_{ij} + 1}{N_i + n}$$

siendo $n$ el número de valores que puede tomar el atributo $A_j$.

En cambio, para los atributos numéricos, se asume que estos siguen una distribución normal, por lo que será necesario calcular la media y la desvicación típica de cada atributo para cada clase:

$$\mu_{ij} = \frac{\sum_{k=1}^{N_i} A_{ijk}}{N_i}$$

$$\sigma_{ij} = \sqrt{\frac{\sum_{k=1}^{N_i} (A_{ijk} - \mu_{ij})^2}{N_i}}$$

siendo $A_{ijk}$ el valor del atributo $A_j$ del ejemplo $k$ de la clase $C_i$.

En el método _clasifica()_, se calcula la probabilidad a posteriori de cada clase para el ejemplo a clasificar, y se calcula la clase que maximiza dicha probabilidad con el conjunto de datos de test. Para el caso de los atributos nominales, el cálculo de la probabilidad a posteriori es el siguiente:

$$P(C_i|A_1, ..., A_n) = \frac{P(C_i) \prod_{j=1}^{n} P(A_j|C_i)}{\sum_{k=1}^{m} P(C_k) \prod_{j=1}^{n} P(A_j|C_k)}$$



En cambio, para los atributos numéricos, su verosimilitud se calcula con la función de densidad de la distribución normal con la media y la desviación típica calculadas en el método _entrenamiento()_ y el conjunto de datos de test:

$$P(A_j|C_i) = \frac{1}{\sqrt{2\pi\sigma_{ij}^2}}e^{-\frac{(A_j - \mu_{ij})^2}{2\sigma_{ij}^2}}$$

siendo $\mu_{ij}$ y $\sigma_{ij}$ la media y la desviación típica del atributo $A_j$ para la clase $C_i$.

Para ese cómputo, hemos hecho uso de la función _norm.pdf()_ de la librería _scipy.stats_ que nos permite calcular la función de densidad de la distribución normal. El cálculo de las probabilidades a posteriori se realiza igual que en el caso de los atributos nominales.

Por último, la predicción de la clase se guarda en una lista que se devuelve al final del método y se calcula con la función _argmax()_ de la librería _numpy_ como indica la fórmula del clasificador Naive Bayes.

Una vez tenemos las predicciones, es necesario validar el correcto funcionamiento del clasificador. Por ello, mediante los métodos _validacion()_ y _error()_ de la clase _Clasificador_, se calcula la tasa de fallo del clasificador. Para ello, se compara la clase predicha con la clase real del ejemplo y se calcula la tasa de acierto como el número de aciertos entre el número total de ejemplos y la tasa de error como el complementario de la tasa de acierto:

$$Tasa\ de\ error = 1 - \frac{N_{aciertos}}{N}$$

A continuación, se muestran los resultados obtenidos con el clasificador Naive Bayes propio con los datasets ***Tic-Tac-Toe*** y ***Heart*** para los diferentes tipos de particionado y con y sin corrección de Laplace.

In [1]:
from Datos import Datos
from EstrategiaParticionado import ValidacionSimple, ValidacionCruzada
from Clasificador import ClasificadorNaiveBayes
import numpy as np
from tabulate import tabulate

## Tic-Tac-Toe Dataset

In [2]:
# Cargamos los datos
d1=Datos('./datasets/tic-tac-toe.csv')
d2=Datos('./datasets/heart.csv')

# Creamos el clasificador Naive Bayes (sin correccion de Laplace)
nb=ClasificadorNaiveBayes(laplace=False)

# Validacion simple
vs = ValidacionSimple(3, 0.3)

# Validacion cruzada
vc = ValidacionCruzada(10)

errorvs = nb.validacion(vs, d1, nb)
errorvc = nb.validacion(vc, d1, nb)

In [3]:
# Creamos el clasificador Naive Bayes (con correccion de Laplace)
nb_laplace=ClasificadorNaiveBayes(laplace=True)

# Validacion simple
vs_laplace = ValidacionSimple(3, 0.3)

# Validacion cruzada
vc_laplace = ValidacionCruzada(10)

errorvs_laplace = nb_laplace.validacion(vs_laplace, d1, nb_laplace)
errorvc_laplace = nb_laplace.validacion(vc_laplace, d1, nb_laplace)

In [4]:
# print error comparison between laplace and non-laplace in a table using tabulate
table = [["Validacion Simple", np.mean(errorvs), np.std(errorvs), np.mean(errorvs_laplace), np.std(errorvs_laplace)],
            ["Validacion Cruzada", np.mean(errorvc), np.std(errorvc), np.mean(errorvc_laplace), np.std(errorvc_laplace)]]
print("Tabla de errores - Tic Tac Toe")
print(tabulate(table, headers=["", "Mean", "Std", "Mean (Laplace)", "Std (Laplace)"], tablefmt="fancy_grid"))

Tabla de errores - Tic Tac Toe
╒════════════════════╤══════════╤═══════════╤══════════════════╤═════════════════╕
│                    │     Mean │       Std │   Mean (Laplace) │   Std (Laplace) │
╞════════════════════╪══════════╪═══════════╪══════════════════╪═════════════════╡
│ Validacion Simple  │ 0.340302 │ 0.0156687 │         0.326365 │      0.00914519 │
├────────────────────┼──────────┼───────────┼──────────────────┼─────────────────┤
│ Validacion Cruzada │ 0.346623 │ 0.0419577 │         0.34648  │      0.0382972  │
╘════════════════════╧══════════╧═══════════╧══════════════════╧═════════════════╛


Como podemos observar en la tabla anterior, el clasificador Naive Bayes propio obtiene mejores resultados con el particionado de validación simple que con el particionado de validación cruzada. Esto se debe a que el particionado de validación cruzada divide el conjunto de datos en $k$ subconjuntos de igual tamaño, por lo que el número de ejemplos de cada clase en cada subconjunto puede variar. En cambio, el particionado de validación simple divide el conjunto de datos en dos subconjuntos, uno de entrenamiento y otro de test con un 70% y un 30% de los ejemplos respectivamente, por lo que el número de ejemplos de cada clase en cada subconjunto puede ser más similar.

Por otro lado, los valores obtenidos con la corrección de Laplace son mejores que los obtenidos sin ella. Esto se debe a que, al aplicar la corrección de Laplace, se evita que la probabilidad a posteriori de una clase sea 0, lo que haría que se descartara como posible clase para el ejemplo a clasificar. Con lo cual, al aplicar la corrección de Laplace tenemos en cuenta todos los ejemplos de entrenamiento, lo que hace que el clasificador sea más preciso.

## Heart dataset

In [5]:
# Cargamos los datos
d2=Datos('./datasets/heart.csv')

# Creamos el clasificador Naive Bayes (sin correccion de Laplace)
nb=ClasificadorNaiveBayes(laplace=False)

# Validacion simple
vs = ValidacionSimple(3, 0.3)

# Validacion cruzada
vc = ValidacionCruzada(10)

errorvs = nb.validacion(vs, d1, nb)
errorvc = nb.validacion(vc, d1, nb)

In [6]:
# Creamos el clasificador Naive Bayes (con correccion de Laplace)
nb_laplace=ClasificadorNaiveBayes(laplace=True)

# Validacion simple
vs_laplace = ValidacionSimple(3, 0.3)

# Validacion cruzada
vc_laplace = ValidacionCruzada(10)

errorvs_laplace = nb_laplace.validacion(vs_laplace, d1, nb_laplace)
errorvc_laplace = nb_laplace.validacion(vc_laplace, d1, nb_laplace)

In [7]:
# print error comparison between laplace and non-laplace in a table using tabulate

table = [["Validacion Simple", np.mean(errorvs), np.std(errorvs), np.mean(errorvs_laplace), np.std(errorvs_laplace)],
            ["Validacion Cruzada", np.mean(errorvc), np.std(errorvc), np.mean(errorvc_laplace), np.std(errorvc_laplace)]]
print("Tabla de errores - Heart")
print(tabulate(table, headers=["", "Mean", "Std", "Mean (Laplace)", "Std (Laplace)"], tablefmt="fancy_grid"))

Tabla de errores - Heart
╒════════════════════╤══════════╤═══════════╤══════════════════╤═════════════════╕
│                    │     Mean │       Std │   Mean (Laplace) │   Std (Laplace) │
╞════════════════════╪══════════╪═══════════╪══════════════════╪═════════════════╡
│ Validacion Simple  │ 0.324042 │ 0.0102576 │         0.351916 │       0.0186555 │
├────────────────────┼──────────┼───────────┼──────────────────┼─────────────────┤
│ Validacion Cruzada │ 0.346502 │ 0.0510728 │         0.346502 │       0.0475715 │
╘════════════════════╧══════════╧═══════════╧══════════════════╧═════════════════╛


Según los valores obtenidos, podemos observar que la tasa de error para la validación cruzada es menor que para la validación simple. Esto se debe a que el conjunto de datos es más grande que el del dataset anterior, por lo que el particionado de validación cruzada es más preciso que el de validación simple.

En cambio, este dataset no está compuesto puramente de atributos nominales, sino que también tiene atributos numéricos. Se puede apreciar que los resultados obtenidos con la corrección de Laplace no mejoran los resultados obtenidos sin ella. Podemos deducir que es debido a que las probabilidades posterioris de los atributos numéricos se calculan suponiendo que siguen una distribución normal que no tiene por qué ser cierta, sino que es una aproximación. Por ello, la aplicación de la corrección de Laplace a algunos atributos nominales puede mejorar los resultados, pero no a los atributos numéricos.

# 2. Naive-Bayes con Scikit-Learn

En esta sección compararemos diferentes funciones de Naive Bayes ofrecidas por Sckikit-Learn, para ver cual modelo trabaja de mejor forma con los datasets de Tic-Tac-Toe y Heart. Y haremos comparaciones usando un encoder y diferentes validaciones. Para facilitar las comparaciones utilizaremos tablas, en las cuales podremos observar los datos the puntaje, error y desviación tipica.

Las funciones a utilizar de Scikit-Learn son:

* GaussianNB: Gaussian Naive Bayes.
* MultinomialNB: Naive Bayes multinomial.
* CategoricalNB: Categorical Naive Bayes. 
* train_test_split: Particionado de Validación Simple.
* KFold: Particionado de Validación Cruzada.
* OneHotEncoder: Codificación de atributos nominales.


In [8]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import numpy as np

## Tic-Tac-Toe Dataset

In [9]:
X = d1.datos.iloc[:, :-1].to_numpy()

y = d1.datos.iloc[:, -1].to_numpy()

### GaussianNB, MultinomialNB, CategoricalNB con Validación Simple y Cruzada

In [10]:
# Naive Bayes Gaussiano
gnb = GaussianNB()
gnb_errors = []
gnb_scores = []

num_samples = 3

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    gnb.fit(X_train, y_train)
    accuracy = gnb.score(X_test, y_test)
    gnb_scores.append(accuracy)
    error = 1 - accuracy
    gnb_errors.append(error)

score_gnb = np.mean(gnb_scores)
error_rate_gnb = np.mean(gnb_errors)
std_gnb = np.std(gnb_errors)

# Naive Bayes Multinomial
mnb = MultinomialNB()
mnb_errors = []
mnb_scores = []

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    mnb.fit(X_train, y_train)
    accuracy = mnb.score(X_test, y_test)
    error = 1 - accuracy
    mnb_errors.append(error)
    mnb_scores.append(accuracy)

score_mnb = np.mean(mnb_scores)
error_rate_mnb = np.mean(mnb_errors)
std_mnb = np.std(mnb_errors)

# Naive Bayes Categorico
cnb = CategoricalNB()
cnb_errors = []
cnb_scores = []

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    cnb.fit(X_train, y_train)
    accuracy = cnb.score(X_test, y_test)
    cnb_scores.append(accuracy)
    error = 1 - accuracy
    cnb_errors.append(error)

score_cnb = np.mean(cnb_scores)
error_rate_cnb = np.mean(cnb_errors)
std_cnb = np.std(cnb_errors)
# Uso de validación cruzada

# numero de folds definidos
folds = 10
kf = KFold(n_splits=folds, shuffle=True)

# Crear lista para tener los puntajes de cada fold
accuracy_scores = {"GaussianNB": [], "MultinomialNB": [], "CategoricalNB": []}
# Crear lista para tener los errores de cada fold
error_rates = {"GaussianNB": [], "MultinomialNB": [], "CategoricalNB": []}


# Dividir los datos en train y test y evaluar cada fold
for train_index, test_index in kf.split(X):
    X_train_cv, X_test_cv = X[train_index], X[test_index]
    y_train_cv, y_test_cv = y[train_index], y[test_index]

    # Naive Bayes Gaussiano para validacion cruzada
    gnb_cv = GaussianNB()
    # Multinomial Naive Bayes para validacion cruzada
    mnb_cv = MultinomialNB()
    # Categorical Naive Bayes para validacion cruzada
    cnb_cv = CategoricalNB()

    # Entrenar los modelos
    gnb_cv.fit(X_train_cv, y_train_cv)
    mnb_cv.fit(X_train_cv, y_train_cv)
    cnb_cv.fit(X_train_cv, y_train_cv)

    # prediction
    y_pred_cv_gnb = gnb_cv.predict(X_test_cv)
    y_pred_cv_mnb = mnb_cv.predict(X_test_cv)
    y_pred_cv_cnb = cnb_cv.predict(X_test_cv)

    # Obtener los puntajes
    accuracy_scores["GaussianNB"].append(accuracy_score(y_test_cv, y_pred_cv_gnb))
    accuracy_scores["MultinomialNB"].append(accuracy_score(y_test_cv, y_pred_cv_mnb))
    accuracy_scores["CategoricalNB"].append(accuracy_score(y_test_cv, y_pred_cv_cnb))

    # Obtener los errores
    error_rates["GaussianNB"].append(1-accuracy_score(y_test_cv, y_pred_cv_gnb))
    error_rates["MultinomialNB"].append(1-accuracy_score(y_test_cv, y_pred_cv_mnb))
    error_rates["CategoricalNB"].append(1-accuracy_score(y_test_cv, y_pred_cv_cnb))

# Calcular los promedios de los puntajes, errores y desviaciones estandar
accuracy_score_gnb_cv = np.mean(accuracy_scores["GaussianNB"])
error_rate_gnb_cv = np.mean(error_rates["GaussianNB"])
std_gnb_cv = np.std(error_rates["GaussianNB"])

accuracy_score_mnb_cv = np.mean(accuracy_scores["MultinomialNB"])
error_rate_mnb_cv = np.mean(error_rates["MultinomialNB"])
std_mnb_cv = np.std(error_rates["MultinomialNB"])

accuracy_score_cnb_cv = np.mean(accuracy_scores["CategoricalNB"])
error_rate_cnb_cv = np.mean(error_rates["CategoricalNB"])
std_cnb_cv = np.std(error_rates["CategoricalNB"])

# Crear tabla de comparación de resultados
table = [["GaussianNB", score_gnb, error_rate_gnb, std_gnb],
            ["MultinomialNB", score_mnb, error_rate_mnb, std_mnb],
            ["CategoricalNB", score_cnb, error_rate_cnb, std_cnb],
            ["GaussianNB (CV)", accuracy_score_gnb_cv, error_rate_gnb_cv, std_gnb_cv],
            ["MultinomialNB (CV)", accuracy_score_mnb_cv, error_rate_mnb_cv, std_mnb_cv],
            ["CategoricalNB (CV)", accuracy_score_cnb_cv, error_rate_cnb_cv, std_cnb_cv]]

print("Tabla de Sklearn - Tic-Tac Toe")
print(tabulate(table, headers=["", "Score", "Error Rate", "Std"], tablefmt="fancy_grid"))

Tabla de Sklearn - Tic-Tac Toe
╒════════════════════╤══════════╤══════════════╤════════════╕
│                    │    Score │   Error Rate │        Std │
╞════════════════════╪══════════╪══════════════╪════════════╡
│ GaussianNB         │ 0.717593 │     0.282407 │ 0.00911344 │
├────────────────────┼──────────┼──────────────┼────────────┤
│ MultinomialNB      │ 0.665509 │     0.334491 │ 0.0163682  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ CategoricalNB      │ 0.695602 │     0.304398 │ 0.0360101  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ GaussianNB (CV)    │ 0.712982 │     0.287018 │ 0.0264506  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ MultinomialNB (CV) │ 0.657621 │     0.342379 │ 0.0172653  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ CategoricalNB (CV) │ 0.697292 │     0.302708 │ 0.0416086  │
╘════════════════════╧══════════╧══════════════╧════════════╛


En la siguiente tabla se puede observar como para el dataset de Tic Tac Toe, con una validación simple se obtiene un porcentaje de error bastantes parecidos. No hay mucha variación en las predicciones de cada fold (Validación Cruzada) o iteración (Validación Simple). 

El modelo que esta sirviendo de mejor forma es el GaussianNB, con porcentajes de acierto de alrededor del 70%. Esto se debe principalmente que todos los datos nominales los cambiamos a numeros, lo cual permite al modelo sacar su mejor versión.

## Usar OneHotEncoder para el mismo dataset y encontrar las diferencias

In [11]:
# Hacer la transformación con el OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_hot_encoded = enc.fit_transform(X)

### GaussianNB, MultinomialNB, CategoricalNB con Validación Simple y Cruzada

In [12]:
# Naive Bayes Gaussiano
gnb_enc = GaussianNB()

enc_scores_gnb = []
enc_errors_gnb = []

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X_hot_encoded, y, test_size=0.3)
    gnb_enc.fit(X_train, y_train)
    accuracy = gnb_enc.score(X_test, y_test)
    enc_scores_gnb.append(accuracy)
    enc_errors_gnb.append(1 - accuracy)

gnb_enc_score = np.mean(enc_scores_gnb)
gnb_enc_error_rate = np.mean(enc_errors_gnb)
gnb_enc_std = np.std(enc_errors_gnb)

# Naive Bayes Multinomial
mnb_enc = MultinomialNB()

enc_scores_mnb = []
enc_errors_mnb = []

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X_hot_encoded, y, test_size=0.3)
    mnb_enc.fit(X_train, y_train)
    accuracy = mnb_enc.score(X_test, y_test)
    enc_scores_mnb.append(accuracy)
    enc_errors_mnb.append(1 - accuracy)

mnb_enc_score = np.mean(enc_scores_mnb)
mnb_enc_error_rate = np.mean(enc_errors_mnb)
mnb_enc_std = np.std(enc_errors_mnb)

# Naive Bayes Categorico
cnb_enc = CategoricalNB()

enc_scores_cnb = []
enc_errors_cnb = []

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X_hot_encoded, y, test_size=0.3)
    cnb_enc.fit(X_train, y_train)
    accuracy = cnb_enc.score(X_test, y_test)
    enc_scores_cnb.append(accuracy)
    enc_errors_cnb.append(1 - accuracy)

cnb_enc_score = np.mean(enc_scores_cnb)
cnb_enc_error_rate = np.mean(enc_errors_cnb)
cnb_enc_std = np.std(enc_errors_cnb)

# Hacer Validación Cruzada

# Definir el numero de folds
folds = 10

# Crear el objeto KFold
kf = KFold(n_splits=folds, shuffle=True)

# Crear diccioanrio para accuracy scores
accuracy_scores_enc = {"GaussianNB": [], "MultinomialNB": [], "CategoricalNB": []}
# Crear diccionario para errores
error_rates_enc = {"GaussianNB": [], "MultinomialNB": [], "CategoricalNB": []}

# Dividir folds entre train y test y evaluar cada uno
for train_index, test_index in kf.split(X_hot_encoded):
    X_train_cv_enc, X_test_cv_enc = X_hot_encoded[train_index], X_hot_encoded[test_index]
    y_train_cv_enc, y_test_cv_enc = y[train_index], y[test_index]

    # Crear modelo de Gaussian Naive Bayes
    gnb_cv_enc = GaussianNB()

    # Crear modelo de Multinomial Naive Bayes
    mnb_cv_enc = MultinomialNB()

    # Crear modelo de Categorical Naive Bayes
    cnb_cv_enc = CategoricalNB()

    # Entrenar los modelos
    gnb_cv_enc.fit(X_train_cv_enc, y_train_cv_enc)
    mnb_cv_enc.fit(X_train_cv_enc, y_train_cv_enc)
    cnb_cv_enc.fit(X_train_cv_enc, y_train_cv_enc)

    # Predecir
    y_pred_cv_enc_gnb = gnb_cv_enc.predict(X_test_cv_enc)
    y_pred_cv_enc_mnb = mnb_cv_enc.predict(X_test_cv_enc)
    y_pred_cv_enc_cnb = cnb_cv_enc.predict(X_test_cv_enc)

    # Obtener puntajes
    accuracy_scores_enc["GaussianNB"].append(accuracy_score(y_test_cv_enc, y_pred_cv_enc_gnb))
    accuracy_scores_enc["MultinomialNB"].append(accuracy_score(y_test_cv_enc, y_pred_cv_enc_mnb))
    accuracy_scores_enc["CategoricalNB"].append(accuracy_score(y_test_cv_enc, y_pred_cv_enc_cnb))

    # Obtener Errores
    error_rates_enc["GaussianNB"].append(1-accuracy_score(y_test_cv_enc, y_pred_cv_enc_gnb))
    error_rates_enc["MultinomialNB"].append(1-accuracy_score(y_test_cv_enc, y_pred_cv_enc_mnb))
    error_rates_enc["CategoricalNB"].append(1-accuracy_score(y_test_cv_enc, y_pred_cv_enc_cnb))

# Calcular promedios de puntajes, errores y desviaciones estandar
accuracy_score_gnb_enc = np.mean(accuracy_scores_enc["GaussianNB"])
error_rate_gnb_enc = np.mean(error_rates_enc["GaussianNB"])
std_gnb_enc = np.std(error_rates_enc["GaussianNB"])

accuracy_score_mnb_enc = np.mean(accuracy_scores_enc["MultinomialNB"])
error_rate_mnb_enc = np.mean(error_rates_enc["MultinomialNB"])
std_mnb_enc = np.std(error_rates_enc["MultinomialNB"])

accuracy_score_cnb_enc = np.mean(accuracy_scores_enc["CategoricalNB"])
error_rate_cnb_enc = np.mean(error_rates_enc["CategoricalNB"])
std_cnb_enc = np.std(error_rates_enc["CategoricalNB"])


# Crear tabla de comparación de resultados
table = [["GaussianNB", gnb_enc_score, gnb_enc_error_rate, gnb_enc_std],
            ["MultinomialNB", mnb_enc_score, mnb_enc_error_rate, mnb_enc_std],
            ["CategoricalNB", cnb_enc_score, cnb_enc_error_rate, cnb_enc_std],
            ["GaussianNB (CV)", accuracy_score_gnb_enc, error_rate_gnb_enc, std_gnb_enc],
            ["MultinomialNB (CV)", accuracy_score_mnb_enc, error_rate_mnb_enc, std_mnb_enc],
            ["CategoricalNB (CV)", accuracy_score_cnb_enc, error_rate_cnb_enc, std_cnb_enc]]

print("Tabla de Scikit Learn con HotEncoder - Tic-Tac Toe")
print(tabulate(table, headers=["", "Score", "Error Rate", "Std"], tablefmt="fancy_grid"))



Tabla de Scikit Learn con HotEncoder - Tic-Tac Toe
╒════════════════════╤══════════╤══════════════╤════════════╕
│                    │    Score │   Error Rate │        Std │
╞════════════════════╪══════════╪══════════════╪════════════╡
│ GaussianNB         │ 0.68287  │     0.31713  │ 0.00433062 │
├────────────────────┼──────────┼──────────────┼────────────┤
│ MultinomialNB      │ 0.719907 │     0.280093 │ 0.00163682 │
├────────────────────┼──────────┼──────────────┼────────────┤
│ CategoricalNB      │ 0.668981 │     0.331019 │ 0.0216531  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ GaussianNB (CV)    │ 0.667917 │     0.332083 │ 0.0451438  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ MultinomialNB (CV) │ 0.697127 │     0.302873 │ 0.0502824  │
├────────────────────┼──────────┼──────────────┼────────────┤
│ CategoricalNB (CV) │ 0.689814 │     0.310186 │ 0.0474748  │
╘════════════════════╧══════════╧══════════════╧════════════╛


En la siguiente tabla se puede observar como para el dataset de Tic Tac Toe, con una validación simple se obtienen porcentajes de error parecidos a los de la validación cruzada. Cuando usamos el OneHotEncoder nos fijamos que los tres modelos tienen comportamientos muy similares, donde los tres porcentajes de acierto finales (usando validación simple) rondan el 70%, siendo el modelo MultinomialNB el que obtiene un porcentaje de acierto mas alto. 

Cuando le aplicamos el OneHotEncoder los datos siguen siendo bastante parecidos a cuando no lo aplicamos. Las predicciones que menos varian son las de los modelos GaussianNB y MultinomialNB.

## Heart Dataset


In [13]:
X, y = d2.datos.iloc[:, :-1].to_numpy(), d2.datos.iloc[:, -1].to_numpy()

### GaussianNB con Validación Simple y Cruzada

In [14]:
# Naive Bayes Gaussiano
gnb_heart = GaussianNB()

gnb_heart_scores = []
gnb_heart_errors = []

for i in range(num_samples):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    gnb_heart.fit(X_train, y_train)
    accuracy = gnb_heart.score(X_test, y_test)
    gnb_heart_scores.append(accuracy)
    gnb_heart_errors.append(1 - accuracy)


gnb_heart_score = np.mean(gnb_heart_scores)
gnb_heart_error_rate = np.mean(gnb_heart_errors)
gnb_heart_std = np.std(gnb_heart_errors)

# Validación cruzada

# Definir numero de folds
folds = 10

# Crear objeto KFold
kf_heart = KFold(n_splits=folds, shuffle=True)

# Crear lista para accuracy scores
accuracy_scores_heart = {"GaussianNB": []}

# Crear lista para errores
error_rates_heart = {"GaussianNB": []}

# Dividir los datos en train y test y evaluar cada fold
for train_index, test_index in kf.split(X):

    X_train_cv_heart, X_test_cv_heart = X[train_index], X[test_index]
    y_train_cv_heart, y_test_cv_heart = y[train_index], y[test_index]

    # Crear modelo de Gaussian Naive Bayes
    gnb_cv_heart = GaussianNB()

    # Entrenar el modelo
    gnb_cv_heart.fit(X_train_cv_heart, y_train_cv_heart)

    # Predecir
    y_pred_cv_heart_gnb = gnb_cv_heart.predict(X_test_cv_heart)

    # Obtenemos puntajes
    accuracy_scores_heart["GaussianNB"].append(accuracy_score(y_test_cv_heart, y_pred_cv_heart_gnb))

    # Obtenemos errores
    error_rates_heart["GaussianNB"].append(1-accuracy_score(y_test_cv_heart, y_pred_cv_heart_gnb))


# Calcular promedios de puntajes, errores y desviaciones estandar
accuracy_score_gnb_heart = np.mean(accuracy_scores["GaussianNB"])
error_rate_gnb_heart = np.mean(error_rates["GaussianNB"])
std_gnb_heart = np.std(error_rates["GaussianNB"])

# Crear tabla de comparación de resultados
table = [["GaussianNB", gnb_heart_score, gnb_heart_error_rate, gnb_heart_std], 
            ["GaussianNB CV", accuracy_score_gnb_heart, error_rate_gnb_heart, std_gnb_heart]]

print("Tabla de Sklearn Validación Simple - Heart")
print(tabulate(table, headers=["", "Score", "Error Rate", "Std"], tablefmt="fancy_grid"))

Tabla de Sklearn Validación Simple - Heart
╒═══════════════╤══════════╤══════════════╤═══════════╕
│               │    Score │   Error Rate │       Std │
╞═══════════════╪══════════╪══════════════╪═══════════╡
│ GaussianNB    │ 0.847826 │     0.152174 │ 0.0242149 │
├───────────────┼──────────┼──────────────┼───────────┤
│ GaussianNB CV │ 0.712982 │     0.287018 │ 0.0264506 │
╘═══════════════╧══════════╧══════════════╧═══════════╛


Para este dataset, solo usamos el modelo GaussianNB, ya que es el único que nos permite trabajar con atributos numéricos. MultinomialNB y CategoricalNB trabajan con atributos nominales, por lo tanto su uso no es adecuado para este dataset. Para poder hacer uso de estos modelos, sería necesario discretizar los atributos numéricos, pero con ello perderíamos información y precisión.

En la tabla, podemos observar como la tasa de error es mucho menor usando el tipo de particionado de Validación Simple que con el de Validación Cruzada. Esto se debe a que el dataset no es muy grande, y al hacer el particionado en k folds, podemos estar teniendo muchos sesgos de información.

### Usar HotEncoder para el mismo dataset y revisar las diferencias

In [15]:
X_hot_encoded = enc.fit_transform(X)

#### GaussianNB

In [16]:
# Naive Bayes Gaussiano
gnb_heart_enc = GaussianNB()

gnb_heart_enc_scores = []
gnb_heart_enc_errors = []

for i in range(num_samples):

    X_train, X_test, y_train, y_test = train_test_split(X_hot_encoded, y, test_size=0.3)

    gnb_heart_enc.fit(X_train, y_train)

    accuracy = gnb_heart_enc.score(X_test, y_test)

    gnb_heart_enc_scores.append(accuracy)

    gnb_heart_enc_errors.append(1 - accuracy)

gnb_heart_enc_score = np.mean(gnb_heart_enc_scores)
gnb_heart_enc_error_rate = np.mean(gnb_heart_enc_errors)
gnb_heart_enc_std = np.std(gnb_heart_enc_errors)

# Hacer validación cruzada

# Definir numero de folds
folds = 10

# Crear objeto KFold
kf = KFold(n_splits=folds, shuffle=True)

# Crear diccionario para puntajes
accuracy_scores_heart_enc = {"GaussianNB": []}

# Crear diccionario para errores
error_rates_heart_enc = {"GaussianNB": []}


# Dividir los datos en train y test y evaluar cada fold
for train_index, test_index in kf.split(X_hot_encoded):
    X_train_cv_heart_enc, X_test_cv_heart_enc = X_hot_encoded[train_index], X_hot_encoded[test_index]
    y_train_cv_heart_enc, y_test_cv_heart_enc = y[train_index], y[test_index]

    # Crear modelo de Gaussian Naive Bayes
    gnb_cv_heart_enc = GaussianNB()

    # Entrenar el modelo
    gnb_cv_heart_enc.fit(X_train_cv_heart_enc, y_train_cv_heart_enc)

    # Predecir
    y_pred_cv_heart_enc_gnb = gnb_cv_heart_enc.predict(X_test_cv_heart_enc)

    # Obtenemos puntajes
    accuracy_scores_heart_enc["GaussianNB"].append(accuracy_score(y_test_cv_heart_enc, y_pred_cv_heart_enc_gnb))

    # Obtenemos errores
    error_rates_heart_enc["GaussianNB"].append(1-accuracy_score(y_test_cv_heart_enc, y_pred_cv_heart_enc_gnb))

# Calcular promedios de puntajes, errores y desviaciones estandar
accuracy_score_gnb_heart_enc = np.mean(accuracy_scores_heart_enc["GaussianNB"])
error_rate_gnb_heart_enc = np.mean(error_rates_heart_enc["GaussianNB"])
std_gnb_heart_enc = np.std(error_rates_heart_enc["GaussianNB"])

# Crear tabla de comparación de resultados
table = [["GaussianNB", gnb_heart_enc_score, gnb_heart_enc_error_rate, gnb_heart_enc_std],
            ["GaussianNB (CV)", accuracy_score_gnb_heart_enc, error_rate_gnb_heart_enc, std_gnb_heart_enc]]

print("Tabla de Sklearn con HotEncoder - Heart")
print(tabulate(table, headers=["", "Score", "Error Rate", "Std"], tablefmt="fancy_grid"))

Tabla de Sklearn con HotEncoder - Heart
╒═════════════════╤══════════╤══════════════╤═══════════╕
│                 │    Score │   Error Rate │       Std │
╞═════════════════╪══════════╪══════════════╪═══════════╡
│ GaussianNB      │ 0.555556 │     0.444444 │ 0.0151809 │
├─────────────────┼──────────┼──────────────┼───────────┤
│ GaussianNB (CV) │ 0.559854 │     0.440146 │ 0.0630948 │
╘═════════════════╧══════════╧══════════════╧═══════════╛


Al aplicar OneHotEncoder, la tasa de error disminuye para los dos tipos de estrategia de particionado, tanto en la Validación Simple y Validación Cruzada. Podríamos considerar el clasificador GaussianNB con Validación Simple, como el más preciso al tener la menor desviación. Aun así, las dos estrategias comparten la misma tasa de fallo que apenas supera el 55%.

Comparado con la tabla anterior en la cual no se hace uso de la codificación, la tasa de fallo es notablemente alta. Podemos concluir que el uso del Encoder no mejora los resultados y se podría despreciar su uso.

# 3. Conclusión

Hemos podido comprobar en esta práctica las diferencias de implementación entre nuestro clasificador Naive Bayes y el de la librería scikit-learn. Nuestro propio clasificador no consta de las mismas funcionalidades que los métodos implementados en la librería scikit-learn pero los resultados obtenidos son ligeramente parecidas.

En algunos casos, el particionado Validación Simple obtiene mejores resultados que el de Validación Cruzada, pero en otros casos ocurre lo contrario. Esto se debe a que el particionado de Validación Cruzada divide el conjunto de datos en $k$ subconjuntos de igual tamaño, por lo que el número de ejemplos de cada clase en cada subconjunto puede variar. En cambio, el particionado de Validación Simple divide el conjunto de datos en dos subconjuntos, uno de entrenamiento y otro de test con un 70% y un 30% de los ejemplos respectivamente, por lo que el número de ejemplos de cada clase en cada subconjunto puede ser más similar.

Por otro lado, la aplicación de la corrección de Laplace mejora los resultados obtenidos por nuestro propio clasificador tal y como indica la teoría pero en algunos casos, su uso puede ser despreciado ya que no es una mejora significativa.

En cuanto a los resultados de Scikit-Learn, debido a que su nivel de exactitud y complejidad es mayor, los resultados obtenidos son mucho más precisos que los nuestros y por ende más fiables. Las diferencias entre la Validación Simple y la Cruzada son las mismas que en nuestro clasificador, pero en este caso, la diferencia de error es mucho menor.

Las diferentes pruebas realizadas con los clasificadores de Scikit-Learn nos han permitido entender las propiedades de cada uno, así como sus punto fuertes y débiles en relación al procesamiento de los datos dependiendo de si era de tipo categórico o numérico.

Por último, se han aplicado estrategias de codificación que han variado los resultados para estos clasificadores, en algunos casos mejorando las predicciones y en otros empeorándolas. Esto es debido a que la codificación de los datos puede hacer que se pierda información, sobre todo, para los atributos numéricos, ya que estos se convierten en atributos nominales.